In [1]:
import sys
sys.path.insert(0, '/cndd/fangming/CEMBA/snmcseq_dev')

from __init__ import *
from __init__jupyterlab import *
import h5py
from scipy import sparse
import pickle
import snmcseq_utils

# get promoter bed file

In [69]:
f = '/datasets/Public_Datasets/BICCN/snmCSeq/References/Annotation/gencode.vM16.annotation_genes_19x.tsv'
gtf = pd.read_csv(f, sep='\t')
gtf

,gene_id,gene_name,chr,start,end,strand,gene_type
0,ENSMUSG00000102693.1,4933401J01Rik,chr1,3073253,3074322,+,TEC
1,ENSMUSG00000064842.1,Gm26206,chr1,3102016,3102125,+,snRNA
2,ENSMUSG00000051951.5,Xkr4,chr1,3205901,3671498,-,protein_coding
3,ENSMUSG00000102851.1,Gm18956,chr1,3252757,3253236,+,processed_pseudogene
4,ENSMUSG00000103377.1,Gm37180,chr1,3365731,3368549,-,TEC
...,...,...,...,...,...,...,...
51767,ENSMUSG00000086695.1,Gm15247,chrX,169985801,169986939,-,antisense_RNA
51768,ENSMUSG00000095562.7,Gm21887,chrX,170009659,170019281,+,protein_coding
51769,ENSMUSG00000093806.1,Asmt,chrX,170672644,170678054,+,protein_coding
51770,ENSMUSG00000096661.1,Gm15069,chrX,170746836,170747207,-,processed_pseudogene


In [73]:
promos = gtf.copy()

cond = promos['strand']=='+'
promos.loc[cond, 'end'] = promos.loc[cond, 'start']
promos.loc[cond, 'start'] = promos.loc[cond, 'start'] - 500

cond = promos['strand']=='-'
promos.loc[cond, 'start'] = promos.loc[cond, 'end'] 
promos.loc[cond, 'end'] = promos.loc[cond, 'end'] + 500

promos

,gene_id,gene_name,chr,start,end,strand,gene_type
0,ENSMUSG00000102693.1,4933401J01Rik,chr1,3072753,3073253,+,TEC
1,ENSMUSG00000064842.1,Gm26206,chr1,3101516,3102016,+,snRNA
2,ENSMUSG00000051951.5,Xkr4,chr1,3671498,3671998,-,protein_coding
3,ENSMUSG00000102851.1,Gm18956,chr1,3252257,3252757,+,processed_pseudogene
4,ENSMUSG00000103377.1,Gm37180,chr1,3368549,3369049,-,TEC
...,...,...,...,...,...,...,...
51767,ENSMUSG00000086695.1,Gm15247,chrX,169986939,169987439,-,antisense_RNA
51768,ENSMUSG00000095562.7,Gm21887,chrX,170009159,170009659,+,protein_coding
51769,ENSMUSG00000093806.1,Asmt,chrX,170672144,170672644,+,protein_coding
51770,ENSMUSG00000096661.1,Gm15069,chrX,170747207,170747707,-,processed_pseudogene


In [74]:
promos['region'] = promos['chr']+"_"+promos['start'].astype(str)+"_"+promos['end'].astype(str)
print(promos['region'].unique().shape, promos.shape)
promos = promos.groupby('region').first().reset_index()
promos['gid'] = promos['gene_id'].apply(lambda x: x.split('.')[0])

print(promos['region'].unique().shape, promos['gid'].unique().shape, promos.shape)
promos

(51740,) (51772, 8)
(51740,) (51740,) (51740, 9)


,region,gene_id,gene_name,chr,start,end,strand,gene_type,gid
0,chr10_100015130_100015630,ENSMUSG00000019966.18,Kitl,chr10,100015130,100015630,+,protein_coding,ENSMUSG00000019966
1,chr10_100139001_100139501,ENSMUSG00000096586.1,Gm22918,chr10,100139001,100139501,+,miRNA,ENSMUSG00000096586
2,chr10_100143754_100144254,ENSMUSG00000095539.1,Gm25287,chr10,100143754,100144254,+,miRNA,ENSMUSG00000095539
3,chr10_100144126_100144626,ENSMUSG00000112831.1,AC167229.2,chr10,100144126,100144626,-,lincRNA,ENSMUSG00000112831
4,chr10_100204342_100204842,ENSMUSG00000112633.1,AC167229.1,chr10,100204342,100204842,-,TEC,ENSMUSG00000112633
...,...,...,...,...,...,...,...,...,...
51735,chrX_99471273_99471773,ENSMUSG00000034403.16,Pja1,chrX,99471273,99471773,-,protein_coding,ENSMUSG00000034403
51736,chrX_99687774_99688274,ENSMUSG00000082505.1,Gm14816,chrX,99687774,99688274,-,processed_pseudogene,ENSMUSG00000082505
51737,chrX_99795971_99796471,ENSMUSG00000083682.1,Pcna-ps1,chrX,99795971,99796471,+,processed_pseudogene,ENSMUSG00000083682
51738,chrX_99820521_99821021,ENSMUSG00000071719.2,Tmem28,chrX,99820521,99821021,+,protein_coding,ENSMUSG00000071719


In [75]:
promos_out = promos[['chr', 'start', 'end', 'gid', 'gene_name', 'strand']]
print(promos_out.shape)
fout = '../../../cicero/gencode_vM16_genes_19x_promoters_500.bed'
# promos_out.to_csv(fout, sep='\t', header=False, index=False)
promos_out.head()

(51740, 6)


,chr,start,end,gid,gene_name,strand
0,chr10,100015130,100015630,ENSMUSG00000019966,Kitl,+
1,chr10,100139001,100139501,ENSMUSG00000096586,Gm22918,+
2,chr10,100143754,100144254,ENSMUSG00000095539,Gm25287,+
3,chr10,100144126,100144626,ENSMUSG00000112831,AC167229.2,-
4,chr10,100204342,100204842,ENSMUSG00000112633,AC167229.1,-


In [76]:
!head $fout

chr10	100015130	100015630	ENSMUSG00000019966	Kitl	+
chr10	100139001	100139501	ENSMUSG00000096586	Gm22918	+
chr10	100143754	100144254	ENSMUSG00000095539	Gm25287	+
chr10	100144126	100144626	ENSMUSG00000112831	AC167229.2	-
chr10	100204342	100204842	ENSMUSG00000112633	AC167229.1	-
chr10	100298652	100299152	ENSMUSG00000112245	Gm8689	-
chr10	100307193	100307693	ENSMUSG00000112069	Gm9476	-
chr10	100320270	100320770	ENSMUSG00000112522	AC164629.4	-
chr10	100320602	100321102	ENSMUSG00000112178	AC164629.2	+
chr10	100321425	100321925	ENSMUSG00000112122	AC164629.1	-


# PMAT(promoters) counting
`/cndd2/fangming/projects/scf_enhancers/data/atac_raw_forpromoters`

`gencode_vM16_genes_19x_promoters_500.bed`

In [55]:
datadir = '/cndd2/fangming/projects/scf_enhancers/data/atac_raw_forpromoters/'

In [57]:
# # delete all PM
# fp = os.path.join(datadir, '*.snap')
# fs = glob.glob(fp)
# session = 'PM'
# print(fs)

# for f in fs:
#     print(f)
    
#     with h5py.File(f,  "a") as fh:
#         print(fh.keys())
#         if session in list(fh.keys()):
#             print('{} exists, to remove'.format(session))
#             del fh['PM']
            
#     with h5py.File(f,  "r") as fh:
#         print(fh.keys())


['/cndd2/fangming/projects/scf_enhancers/data/atac_raw_forpromoters/CEMBA180612_5D.snap', '/cndd2/fangming/projects/scf_enhancers/data/atac_raw_forpromoters/CEMBA171212_4B.snap', '/cndd2/fangming/projects/scf_enhancers/data/atac_raw_forpromoters/CEMBA180104_4B.snap', '/cndd2/fangming/projects/scf_enhancers/data/atac_raw_forpromoters/CEMBA171213_4B.snap', '/cndd2/fangming/projects/scf_enhancers/data/atac_raw_forpromoters/CEMBA180410_2C.snap', '/cndd2/fangming/projects/scf_enhancers/data/atac_raw_forpromoters/CEMBA171207_3C.snap', '/cndd2/fangming/projects/scf_enhancers/data/atac_raw_forpromoters/CEMBA180409_2C.snap', '/cndd2/fangming/projects/scf_enhancers/data/atac_raw_forpromoters/CEMBA180618_5D.snap', '/cndd2/fangming/projects/scf_enhancers/data/atac_raw_forpromoters/CEMBA171206_3C.snap']
/cndd2/fangming/projects/scf_enhancers/data/atac_raw_forpromoters/CEMBA180612_5D.snap
<KeysViewHDF5 ['AM', 'BD', 'FM', 'GM', 'HD']>
/cndd2/fangming/projects/scf_enhancers/data/atac_raw_forpromoters/

In [39]:
# run add_pmat (in scripts)

# Reformat PMAT(promoters) counts
- add back former PM code 

In [96]:
datadir = '/cndd2/fangming/projects/scf_enhancers/data/atac_raw_forpromoters/'

In [82]:
fp = os.path.join(datadir, '*.snap')
fs = glob.glob(fp)
print(fs)

for f in fs:
    with h5py.File(f,  "r") as fh:
        print(fh.keys())

['/cndd2/fangming/projects/scf_enhancers/data/atac_raw_forpromoters/CEMBA180612_5D.snap', '/cndd2/fangming/projects/scf_enhancers/data/atac_raw_forpromoters/CEMBA171212_4B.snap', '/cndd2/fangming/projects/scf_enhancers/data/atac_raw_forpromoters/CEMBA180104_4B.snap', '/cndd2/fangming/projects/scf_enhancers/data/atac_raw_forpromoters/CEMBA171213_4B.snap', '/cndd2/fangming/projects/scf_enhancers/data/atac_raw_forpromoters/CEMBA180410_2C.snap', '/cndd2/fangming/projects/scf_enhancers/data/atac_raw_forpromoters/CEMBA171207_3C.snap', '/cndd2/fangming/projects/scf_enhancers/data/atac_raw_forpromoters/CEMBA180409_2C.snap', '/cndd2/fangming/projects/scf_enhancers/data/atac_raw_forpromoters/CEMBA180618_5D.snap', '/cndd2/fangming/projects/scf_enhancers/data/atac_raw_forpromoters/CEMBA171206_3C.snap']
<KeysViewHDF5 ['AM', 'BD', 'FM', 'GM', 'HD', 'PM']>
<KeysViewHDF5 ['AM', 'BD', 'FM', 'GM', 'HD', 'PM']>
<KeysViewHDF5 ['AM', 'BD', 'FM', 'GM', 'HD', 'PM']>
<KeysViewHDF5 ['AM', 'BD', 'FM', 'GM', 'HD

In [97]:
f = os.path.join(datadir, 'CEMBA171206_3C.snap')
with h5py.File(f, 'r') as fh:
    for key, val in fh['BD'].items():
        print(key, val)
    print('...')
    for key, val in fh['PM'].items():
        print(key, val)
    print('...')
    
    print(fh['PM']['idx'].shape)
    print(np.array(fh['PM']['idx']).max()) # cell
    print(np.array(fh['PM']['idy']).max()) # gene
    print(fh['PM']['peakChrom'].shape)
    print(fh['PM']['peakStart'].shape)
    print(fh['PM']['peakEnd'].shape)

CM <HDF5 dataset "CM": shape (15731,), type "<u4">
PE <HDF5 dataset "PE": shape (15731,), type "<u4">
PL <HDF5 dataset "PL": shape (15731,), type "<u4">
PP <HDF5 dataset "PP": shape (15731,), type "<u4">
SA <HDF5 dataset "SA": shape (15731,), type "<u4">
SE <HDF5 dataset "SE": shape (15731,), type "<u4">
TN <HDF5 dataset "TN": shape (15731,), type "<u4">
UM <HDF5 dataset "UM": shape (15731,), type "<u4">
UQ <HDF5 dataset "UQ": shape (15731,), type "<u4">
US <HDF5 dataset "US": shape (15731,), type "<u4">
name <HDF5 dataset "name": shape (15731,), type "|O">
...
count <HDF5 dataset "count": shape (8741471,), type "|u1">
idx <HDF5 dataset "idx": shape (8741471,), type "<u4">
idy <HDF5 dataset "idy": shape (8741471,), type "<u4">
peakChrom <HDF5 dataset "peakChrom": shape (51740,), type "|O">
peakEnd <HDF5 dataset "peakEnd": shape (51740,), type "<u4">
peakStart <HDF5 dataset "peakStart": shape (51740,), type "<u4">
...
(8741471,)
15731
51740
(51740,)
(51740,)
(51740,)


In [98]:
pmat_all = []
cells_all = []

for f in fs:
    dataset = f.split('/')[-1][:-len('.snap')]
    print(dataset)
    with h5py.File(f, 'r') as fh:
        # get peak by cell matrix (sparse)


        # the coo matrix
        counts = np.array(fh['PM']['count'])
        idx_peak = np.array(fh['PM']['idy']) - 1
        idx_cell = np.array(fh['PM']['idx']) - 1

        # cell (cols)
        cells = np.array(['snatac_gene_{}_'.format(dataset)+name.decode('utf-8') 
                        for name in np.array(fh['BD']['name'])]
                ) 
        ncell = len(cells)

        # peak (rows)
        peak_chrs = np.array(fh['PM']['peakChrom'])
        peak_starts = np.array(fh['PM']['peakStart'])
        peak_ends = np.array(fh['PM']['peakEnd'])
        npeak = len(peak_chrs)

        print(fh['PM']['idx'].shape, 
              np.array(fh['PM']['idx']).min(),
              np.array(fh['PM']['idy']).min(),
             )
        print(ncell, npeak)
        print(peak_chrs.shape, peak_starts.shape, peak_ends.shape)

        pmat = sparse.coo_matrix((counts, (idx_peak, idx_cell)), shape=(npeak, ncell))
        # concat
        pmat_all.append(pmat)
        cells_all.append(cells)

cells_all = np.hstack(cells_all)
print(cells_all.shape)
    
pmat_all = sparse.hstack(pmat_all)
print(pmat_all.shape)
    

CEMBA180612_5D
(8346427,) 1 1
14878 51740
(51740,) (51740,) (51740,)
CEMBA171212_4B
(8728787,) 1 1
15703 51740
(51740,) (51740,) (51740,)
CEMBA180104_4B
(4788017,) 1 1
13689 51740
(51740,) (51740,) (51740,)
CEMBA171213_4B
(8125389,) 1 1
16184 51740
(51740,) (51740,) (51740,)
CEMBA180410_2C
(9416673,) 1 1
15586 51740
(51740,) (51740,) (51740,)
CEMBA171207_3C
(9515983,) 1 1
14687 51740
(51740,) (51740,) (51740,)
CEMBA180409_2C
(6589318,) 1 1
14576 51740
(51740,) (51740,) (51740,)
CEMBA180618_5D
(7900345,) 1 1
14631 51740
(51740,) (51740,) (51740,)
CEMBA171206_3C
(8741471,) 1 1
15731 51740
(51740,) (51740,) (51740,)
(135665,)
(51740, 135665)


In [99]:
# # all pickle files
peak_regions = pd.DataFrame(np.array([peak_chrs, peak_starts, peak_ends]).T, 
                            columns=['chr', 'start', 'end']
                           )
peak_regions['chr'] = peak_regions['chr'].apply(lambda x: x.decode('utf-8'))
peak_regions['region'] = peak_regions['chr']+'_'+peak_regions['start'].astype(str)+'_'+peak_regions['end'].astype(str)
peak_regions = pd.merge(peak_regions, promos[['region', 'gid', 'gene_name']],  on='region')[['chr', 'start', 'end', 'gid', 'gene_name', 'region']]

print(peak_regions.shape)
peak_regions.head()

(51740, 6)


,chr,start,end,gid,gene_name,region
0,chr10,100015130,100015630,ENSMUSG00000019966,Kitl,chr10_100015130_100015630
1,chr10,100139001,100139501,ENSMUSG00000096586,Gm22918,chr10_100139001_100139501
2,chr10,100143754,100144254,ENSMUSG00000095539,Gm25287,chr10_100143754_100144254
3,chr10,100144126,100144626,ENSMUSG00000112831,AC167229.2,chr10_100144126_100144626
4,chr10,100204342,100204842,ENSMUSG00000112633,AC167229.1,chr10_100204342_100204842


In [100]:
cells_all

array(['snatac_gene_CEMBA180612_5D_ACATTGGCAACCAGGTACAAGGATGGATACTA',
       'snatac_gene_CEMBA180612_5D_ACATTGGCAACCAGGTACAAGGATTTCATCCA',
       'snatac_gene_CEMBA180612_5D_ACATTGGCAACCAGGTAGGCTGGTCGAATTCC', ...,
       'snatac_gene_CEMBA171206_3C_TCTCGCGCTTCCATCCTTCATCCAAGGCGAAG',
       'snatac_gene_CEMBA171206_3C_TCTCGCGCTTCCATCCTTGGAAGTCCTATCCT',
       'snatac_gene_CEMBA171206_3C_TCTCGCGCTTCCATCCTTGGAAGTTATAGCCT'],
      dtype='<U59')

In [101]:
pmat_all

<51740x135665 sparse matrix of type '<class 'numpy.uint8'>'
	with 72152410 stored elements in COOrdinate format>

In [102]:
print(pmat_all.shape, cells_all.shape, peak_regions.shape)

(51740, 135665) (135665,) (51740, 6)


In [103]:
output_dir = '/cndd2/fangming/projects/scf_enhancers/data/organized_cell_level/version_nov9'
outputs = [
    'mat_promoters_snatac_gene.pkl',
    'region_promoters_snatac_gene.pkl', 
    'cell_promoters_snatac_gene.pkl',
    ]
output_objs = [
    pmat_all, 
    peak_regions,
    cells_all,
]

for f, obj in zip(outputs, output_objs):
    f = os.path.join(output_dir, f) 
    print(f)
    with open(f, 'wb') as fh:
        pickle.dump(obj, fh)

/cndd2/fangming/projects/scf_enhancers/data/organized_cell_level/version_nov9/mat_promoters_snatac_gene.pkl
/cndd2/fangming/projects/scf_enhancers/data/organized_cell_level/version_nov9/region_promoters_snatac_gene.pkl
/cndd2/fangming/projects/scf_enhancers/data/organized_cell_level/version_nov9/cell_promoters_snatac_gene.pkl


In [104]:
# output_dir = '/cndd2/fangming/projects/scf_enhancers/enhancer_cell_level/atac'

# f = os.path.join(output_dir, 'combined_enh_cell_atac_mat.cell') 
# snmcseq_utils.export_single_textcol(f, cells_all)

# f = os.path.join(output_dir, 'combined_enh_cell_atac_mat.region') 
# peak_regions.to_csv(f, sep='\t', header=True, index=False)

# f = os.path.join(output_dir, 'combined_enh_cell_atac_mat.npz') 
# sparse.save_npz(f, pmat_all)